In [2]:
import yaml
import json
import pandas as pd

from rules_engine import rules_engine
import utils

In [3]:
use_cols = ['name', 'metDivName', 'countyFips', 'geoIdMsa', 'metDivFp', 'smallCounty', 'tracts']
cbsa_cols = ['name', 'metDivName', 'state', 'countyFips', 'county', 'tracts','geoIdMsa', 'metDivFp', 'smallCounty', 
			 'stateCode', 'tractDecimal']
cbsas = pd.read_csv('../dependencies/tract_to_cbsa_2015.txt', usecols=use_cols, delimiter='|', 
					header=None, names=cbsa_cols, dtype=str) #load tract to CBSA data from platform file
cbsas["tractFips"] = cbsas.countyFips + cbsas.tracts
counties = list(cbsas.countyFips)
tracts = list(cbsas.tractFips)

#load schemas for LAR and transmittal sheet
#schemas contain valid enumerations, including NA values, for each field in the dataset
lar_schema_df = pd.DataFrame(json.load(open("../schemas/lar_schema.json", "r")))
ts_schema_df = pd.DataFrame(json.load(open("../schemas/ts_schema.json", "r")))
!pwd

/Users/roellk/Desktop/HMDA/hmda-test-files/python


In [110]:
path = "../edits_files/validity/"
file = "v688_2.txt"
#lar validator checks a dataframe and returns a JSON with generate_error_files
checker = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, counties=counties) 
#load data to checker
ts_df, lar_df = utils.read_data_file(path=path, data_file=file)
checker.load_data_frames(ts_df, lar_df)
for func in dir(checker):
    if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
        #print("applying:", func)
        getattr(checker, func)()
res_df = pd.DataFrame(checker.results)
res_df[(res_df.status=="failed")&(res_df.edit_name==file[:-4])]

,edit_name,fail_count,fields,row_ids,row_type,status
198,v688_2,15.0,Property Value,[Z6L3LVV89BUPEDZZZA0Y8TBLWM2F1DMOYE9THAUVD3U55...,LAR,failed


In [5]:
res_df[res_df.status=="failed"]

,edit_name,fail_count,fields,row_ids,row_type,status
21,v610_2,6.0,app_date,[Z6L3LVV89BUPEDZZZA0Y83RQYGF6MJ18ZFKAJEBYUCP08...,LAR,failed
26,v613_2,15.0,preapproval,[Z6L3LVV89BUPEDZZZA0Y8TBLWM2F1DMOYE9THAUVD3U55...,LAR,failed
120,v656_2,15.0,Type of Purchaser,[Z6L3LVV89BUPEDZZZA0Y8TBLWM2F1DMOYE9THAUVD3U55...,LAR,failed
125,v658_2,4.0,HOEPA,[Z6L3LVV89BUPEDZZZA0Y83RQYGF6MJ18ZFKAJEBYUCP08...,LAR,failed
146,v670_1,1.0,Denial Reason 1,[Z6L3LVV89BUPEDZZZA0YX4O2SUCE0F9QFG634T5NQ3E81],LAR,failed
147,v670_2,1.0,Denial Reason 1,[Z6L3LVV89BUPEDZZZA0YX4O2SUCE0F9QFG634T5NQ3E81],LAR,failed
210,v693_2,6.0,Applicaiton Channel,[Z6L3LVV89BUPEDZZZA0Y83RQYGF6MJ18ZFKAJEBYUCP08...,LAR,failed


In [6]:
midpoint = len(lar_df)/2
lar_df.co_app_eth_1[lar_df.index <= midpoint] = "5"

In [56]:
lar_df[lar_df.points_fees!="NA"]

field,record_id,lei,uli,app_date,loan_type,loan_purpose,preapproval,const_method,occ_type,loan_amount,...,aus_code_5,aus_result_1,aus_result_2,aus_result_3,aus_result_4,aus_result_5,aus_code_16,reverse_mortgage,open_end_credit,business_purpose


In [58]:
lar_df.points_and_fees

AttributeError: 'DataFrame' object has no attribute 'points_and_fees'

In [104]:
lar_df.balloon

0     1
1     1
2     1
3     1
4     2
5     1
6     1
7     1
8     2
9     2
10    1
11    2
12    2
13    1
14    2
Name: balloon, dtype: object